In [1]:
import pandas as pd
import time
import numpy as np
from numba import jit
from sklearn.model_selection import train_test_split
from multiprocessing import *
from sklearn.model_selection import KFold
import gc
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [2]:
start = time.time()
train = pd.read_csv("/home/saurabhg/PuertoSergo/train.csv")
test = pd.read_csv("/home/saurabhg/PuertoSergo/test.csv")
print(time.time()-start)
 
y = train['target']
testid= test['id'].values

train.drop(['id','target'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

8.41864895821


In [3]:
### Drop calc

start = time.time()
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(unwanted, axis=1)  
test = test.drop(unwanted, axis=1)

### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
#train['ps_reg_03_square'] = train['ps_reg_03']*train['ps_reg_03']
#test['ps_reg_03_square'] = test['ps_reg_03']*test['ps_reg_03']
#train['ps_car_14_square'] = train['ps_car_14']*train['ps_car_14']
#test['ps_car_14_square'] = test['ps_car_14']*test['ps_car_14']
def recon(reg):
    integer = int(np.round((40*reg)**2)) 
    for a in range(32):
        if (integer - a) % 31 == 0:
            A = a
    M = (integer - A)//31
    return A, M
train['ps_reg_A'] = train['ps_reg_03'].apply(lambda x: recon(x)[0])
train['ps_reg_M'] = train['ps_reg_03'].apply(lambda x: recon(x)[1])
train['ps_reg_A'].replace(19,-1, inplace=True)
train['ps_reg_M'].replace(51,-1, inplace=True)
test['ps_reg_A'] = test['ps_reg_03'].apply(lambda x: recon(x)[0])
test['ps_reg_M'] = test['ps_reg_03'].apply(lambda x: recon(x)[1])
test['ps_reg_A'].replace(19,-1, inplace=True)
test['ps_reg_M'].replace(51,-1, inplace=True)

print(time.time()-start)


### Froza's baseline
### Froza's baseline
### Froza's baseline
### Froza's baseline

d_median = train.median(axis=0)
d_mean = train.mean(axis=0)
d_skew = train.skew(axis=0)
one_hot = {c: list(train[c].unique()) for c in train.columns if c not in ['id','target']}

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c: #standard arithmetic
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(np.int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(np.int)

    for c in one_hot:
        if len(one_hot[c])>2 and len(one_hot[c]) < 7:
            for val in one_hot[c]:
                df[c+'_oh_' + str(val)] = (df[c].values == val).astype(np.int)
    return df

def multi_transform(df, map=transform_df):
    print('Init Shape: ', df.shape)
    p = Pool(cpu_count())
    df = p.map(transform_df, np.array_split(df, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    print('After Shape: ', df.shape)
    return df

train = multi_transform(train)
test = multi_transform(test)

print(time.time()-start)



63.3370699883
('Init Shape: ', (595212, 39))
('After Shape: ', (595212, 136))
('Init Shape: ', (892816, 39))
('After Shape: ', (892816, 136))
77.0589721203


In [4]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_lgb(act, preds):
#    labels = dtrain.get_label()
    gini_score = gini_normalized(act, preds)
    return 'gini', gini_score,True

In [5]:
MAX_ROUNDS = 5000
OPTIMIZE_ROUNDS = True
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  
f_cats = [f for f in train.columns if "_cat" in f]
# Note: I set EARLY_STOPPING_ROUNDS high so that (when OPTIMIZE_ROUNDS is set)
#       I will get lots of information to make my own judgment.  You should probably
#       reduce EARLY_STOPPING_ROUNDS if you want to do actual early stopping.

In [6]:
n_splits = 10
n_estimators = 1000
folds = KFold(n_splits=n_splits, shuffle=True, random_state=1) 
imp_df = np.zeros((len(train.columns), n_splits))
oof = np.empty(len(train))
sub_preds = np.zeros((len(test),n_splits))
increase = False
np.random.seed(0)
params = {'eta': 0.025, 'max_depth': 4, 
          'subsample': 0.9, 'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':10,
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

In [7]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(y, y)):
    trn_dat, trn_tgt = train.iloc[trn_idx], y.iloc[trn_idx]
    val_dat, val_tgt = train.iloc[val_idx], y.iloc[val_idx]

    clf = lgb.LGBMModel(n_estimators=n_estimators,
                        num_leaves=20,
   #                     boosting_type = 'dart',
                        objective="binary",
                        learning_rate=.025, 
                        subsample=.9, 
                        colsample_bytree=.7,
         #               min_split_gain = .77,
        #                min_child_samples= 500,
         #               is_unbalance = True,
                        min_child_weight =100,
                        reg_alpha=4,
          #              reg_lambda=2,
                      #  nthread=2
                       )
    # Upsample during cross validation to avoid having the same samples
    # in both train and validation sets
    # Validation set is not up-sampled to monitor overfitting
    if increase:
        # Get positive examples
        pos = pd.Series(trn_tgt == 1)
        # Add positive examples
        trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
        trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
        # Shuffle data
        idx = np.arange(len(trn_dat))
        np.random.shuffle(idx)
        trn_dat = trn_dat.iloc[idx]
        trn_tgt = trn_tgt.iloc[idx]
        
    clf.fit(trn_dat, trn_tgt, 
            eval_set=[(trn_dat, trn_tgt), (val_dat, val_tgt)],
            categorical_feature = f_cats,
            eval_metric=gini_lgb,
            early_stopping_rounds=50,
            verbose=100)
    # Find best round for validation setA
    lgb_evals = clf.evals_result_["valid_1"]["gini"]
    #print(clf.evals_result_ )
    # Keep feature importances
    imp_df[:, fold_] = clf.feature_importances_
    #Xgboost provides best round starting from 0 so it has to be incremented
    best_round = np.argsort(lgb_evals)[::-1][0]

    # Predict OOF and submission probas with the best round
    oof[val_idx] = clf.predict(val_dat, num_iteration=best_round)
    # Update submission
    sub_preds[:, fold_] = clf.predict(test, num_iteration=best_round)

    # Display results
    print("Fold %2d : %.6f @%4d / best score is %.6f @%4d"
          % (fold_ + 1,
             gini_normalized(val_tgt, oof[val_idx]),
             n_estimators,
             len(lgb_evals),
             best_round))
          
print("Full OOF score : %.6f" % gini_normalized(y, oof))

Train until valid scores didn't improve in 50 rounds.
[100]	training's gini: 0.264059	valid_1's gini: 0.250618
[200]	training's gini: 0.294919	valid_1's gini: 0.271925
[300]	training's gini: 0.314951	valid_1's gini: 0.278762
[400]	training's gini: 0.329066	valid_1's gini: 0.282926
[500]	training's gini: 0.340329	valid_1's gini: 0.28388
Early stopping, best iteration is:
[495]	training's gini: 0.339882	valid_1's gini: 0.284095
Fold  1 : 0.284081 @1000 / best score is 545.000000 @ 494
Train until valid scores didn't improve in 50 rounds.
[100]	training's gini: 0.262385	valid_1's gini: 0.244912
[200]	training's gini: 0.29334	valid_1's gini: 0.275096
[300]	training's gini: 0.313166	valid_1's gini: 0.288922
[400]	training's gini: 0.326968	valid_1's gini: 0.294137
[500]	training's gini: 0.338644	valid_1's gini: 0.296388
[600]	training's gini: 0.34846	valid_1's gini: 0.297493
Early stopping, best iteration is:
[631]	training's gini: 0.351383	valid_1's gini: 0.297983
Fold  2 : 0.297916 @1000 /

In [11]:

concat_df = pd.DataFrame(sub_preds)
cols = list(map(lambda x: "target_" + str(x), range(len(concat_df.columns))))
concat_df.columns = cols

# Apply ranking, normalization and averaging
concat_df["target"] = (concat_df.rank() / concat_df.shape[0]).mean(axis=1)
concat_df.drop(cols, axis=1, inplace=True)

In [28]:
sub.head(10)

,target
0,0.398860
1,0.299300
2,0.259384
3,0.042534
4,0.586696
5,0.771693
6,0.084905
7,0.774826
8,0.785405
9,0.917545


In [29]:
import scipy
sub = pd.DataFrame(concat_df["target"])
sub['id'] = testid
#sub["target"] = concat_df["target"]


#sub["target"] =scipy.stats.hmean(sub_preds,axis=1)
sub.to_csv("submission_111312017_LGB_.288_10kfold_rank.csv", index=False, float_format="%.9f")